# Modelo 1 - RF baseline

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../models/rf_df_test_cleaned_pred.csv',sep=';', index_col=0)

In [3]:
df.head()

,ind_var30,var36,var15,num_var4,num_var8_0,ind_var13_0,ind_var12_0,saldo_var30,ind_var39_0,num_var30_0,...,num_sal_var16_ult1,delta_imp_reemb_var17_1y3,saldo_medio_var33_hace3,num_op_var40_comer_ult3,num_var29_0,num_aport_var33_ult1,TARGET,ID,prob_0,prob_1
50595,1,1,45,2,0,0,0,781.47,1,3,...,0,0,0.0,0,0,0,0,101019,0.906671,0.093329
5252,1,3,60,2,0,1,0,240003.00,1,6,...,0,0,0.0,0,0,0,0,10518,0.930241,0.069759
21956,1,3,27,1,0,0,0,60.00,1,3,...,0,0,0.0,0,0,0,0,43966,0.943929,0.056071
42714,1,1,38,3,0,0,0,293.70,1,3,...,0,0,0.0,0,0,0,0,85554,0.940258,0.059742
18494,1,3,23,1,0,0,0,3.00,1,3,...,0,0,0.0,0,0,0,0,37070,0.899661,0.100339


prob_0	= probabilidade de estar satisfeito  
prob_1 = probabilidade de churn

In [4]:
def ks(data=None,target=None, prob=None):
    ## finding at: https://www.listendata.com/2019/07/KS-Statistics-Python.html
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 5)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2}'.format).astype(float) * 100
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2}'.format).astype(float) * 100
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2}'.format).astype(float) * 100
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2}'.format).astype(float) * 100
    kstable.index = range(1,6)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    # print(kstable)
    
    #Display KS
    from colorama import Fore
    print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return(kstable)

In [5]:
from scipy.stats import ks_2samp
yhat = df['prob_1']
y = df['TARGET']
def ks_stat(y, yhat):
    return ks_2samp(yhat[y==1], yhat[y!=1]).statistic
ks_stat(y, yhat)

0.45876841159954895

In [6]:
results_quintil = ks(df, target='TARGET', prob='prob_1')
results_quintil

KS is 44.0% at decile 2


,min_prob,max_prob,events,nonevents,event_rate,nonevent_rate,cum_eventrate,cum_noneventrate,KS
Decile,,,,,,,,,
1,0.312993,0.859385,370,2671,61.0,18.0,61.0,18.0,42.7
2,0.125651,0.312718,129,2910,21.0,20.0,82.0,38.0,44.0
3,0.070680,0.125633,70,2972,12.0,20.0,94.0,59.0,35.1
4,0.026147,0.070666,30,3011,4.9,21.0,99.0,79.0,19.5
5,0.000000,0.026141,8,3033,1.3,21.0,100.0,100.0,0.0


In [7]:
from sklearn.metrics import roc_auc_score

print("Roc Auc: ",roc_auc_score(df['TARGET'], df['prob_1']))
print("Gini: ",2*(roc_auc_score(df['TARGET'], df['prob_1']))-1)

Roc Auc:  0.805190331023086
Gini:  0.610380662046172


In [8]:
results_quintil.columns = ['min_prob','max_prob','Qtd de eventos (Target)','Qtd de não-eventos (Target)','% de eventos no Decil','% de não-eventos no Decil','% de evento acumulados','% de não-evento acumulados','KS']
results_quintil.to_csv("results_model1_rfbase.csv",sep=';')

In [9]:
results_quintil

,min_prob,max_prob,Qtd de eventos (Target),Qtd de não-eventos (Target),% de eventos no Decil,% de não-eventos no Decil,% de evento acumulados,% de não-evento acumulados,KS
Decile,,,,,,,,,
1,0.312993,0.859385,370,2671,61.0,18.0,61.0,18.0,42.7
2,0.125651,0.312718,129,2910,21.0,20.0,82.0,38.0,44.0
3,0.070680,0.125633,70,2972,12.0,20.0,94.0,59.0,35.1
4,0.026147,0.070666,30,3011,4.9,21.0,99.0,79.0,19.5
5,0.000000,0.026141,8,3033,1.3,21.0,100.0,100.0,0.0
